# Regresión logística

Ya sabemos cómo funciona una red neuronal y ya sabemos implementarla mediante Pytorch, así que vamos a empezar a ver el ejemplo más sencillo de una red neuronal, la regresión logística.

Importamos la base de datos de precios de casas de Boston

In [1]:
from sklearn import datasets

boston = datasets.load_boston()

/home/maximo.fernandez/miniconda3/envs/cursopytorch/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets incl

Podemos ver qué trae esta base de datos

In [2]:
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

La llave `DESCR` es una descripción de la base de datos

In [3]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Además tiene las llaves `data` y `target` donde se encuentran los datos anteriormente descritos. La llave `feature_names` contiene los numbres de cada una de las características

Así que creamos un dataframe con los datos

In [4]:
import pandas as pd

boston_df = pd.DataFrame(boston['data'], columns=boston['feature_names'])
boston_df['target'] = boston['target']
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


Por último vamos a ver si hay algún dato faltante

In [5]:
boston_df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
target     0
dtype: int64

## Dataset y Dataloader

Creamos el dataset

In [6]:
import torch

class BostonDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        cols = [col for col in dataframe.columns if col != 'target']
        self.parameters = torch.from_numpy(dataframe[cols].values).type(torch.float32)
        self.targets = torch.from_numpy(dataframe['target'].values).type(torch.float32)
        self.targets = self.targets.reshape((len(self.targets), 1))

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        parameters = self.parameters[idx]
        target = self.targets[idx]
        return parameters, target

In [7]:
ds = BostonDataset(boston_df)
len(ds), len(boston_df)

(506, 506)

Para poder entrenar hemos visto que necesitamos dividir los datos en un conjunto de datos de entrenamiento y en un conjunto de datos de validación. Así que dividimos nuestros datos en estos dos conjuntos.

Como no tenemos muchos datos vamos a dividir el conjunto de datos en un 80% para entrenamiento entrenamiento y un 20% para validación

In [8]:
train_ds, valid_ds = torch.utils.data.random_split(ds, [int(0.8*len(ds)), len(ds) - int(0.8*len(ds))], generator=torch.Generator().manual_seed(42))
len(train_ds), len(valid_ds), len(train_ds) + len(valid_ds)

(404, 102, 506)

Vamos a ver una muestra

In [9]:
sample = train_ds[0]
print(f"len(sample): {len(sample)}")

parameters, target = sample
print(f"parameters: {parameters}\ntype parameters: {type(parameters)}\nparameters.dtype: {parameters.dtype}\nparameters.shape: {parameters.shape}\n\n")
print(f"target: {target}, type target: {type(target)}, target.dtype: {target.dtype}, target.shape: {target.shape}")

len(sample): 2
parameters: tensor([3.6150e-02, 8.0000e+01, 4.9500e+00, 0.0000e+00, 4.1100e-01, 6.6300e+00,
        2.3400e+01, 5.1167e+00, 4.0000e+00, 2.4500e+02, 1.9200e+01, 3.9690e+02,
        4.7000e+00])
type parameters: <class 'torch.Tensor'>
parameters.dtype: torch.float32
parameters.shape: torch.Size([13])


target: tensor([27.9000]), type target: <class 'torch.Tensor'>, target.dtype: torch.float32, target.shape: torch.Size([1])


Creamos ahora el dataloader

In [10]:
from torch.utils.data import DataLoader

BS_train = 32
BS_val = 1024

train_dl = DataLoader(train_ds, batch_size=BS_train, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=BS_val, shuffle=False)

Vemos un batch

In [11]:
batch = next(iter(train_dl))
parameters, target = batch[0], batch[1]
type(parameters), parameters.dtype, parameters.shape, type(target), target.shape

(torch.Tensor,
 torch.float32,
 torch.Size([32, 13]),
 torch.Tensor,
 torch.Size([32, 1]))

## Red Neuronal

Creamos una red neuronal para entrenarla

In [12]:
from torch import nn

class BostonNeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_layers=[100, 50, 20]):
        super().__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, hidden_layers[0]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(hidden_layers[0], hidden_layers[1]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(hidden_layers[1], hidden_layers[2]),
            torch.nn.Sigmoid(),
            torch.nn.Linear(hidden_layers[2], num_outputs)
        )

    def forward(self, x):
        logits = self.network(x)
        return logits

Hemos definido una red neuronal genérica, a la que hay que meterle las dimensiones de la capa de entrada y de la capa de salida, y opcionalmente, las dimensiones de las capas ocultas. Así que vamos a ver qué tamaño necesitamos a la entrada y a la salida de la red

Un batch tiene unos parámetros con este tamaño

In [15]:
parameters.shape

torch.Size([32, 13])

Tenemos una matriz de tamaño 32x13. 32 es el tamaño del batch size, mientras que 13 es el número de parámetros, por lo que **a la entrada necesitamos 13 neuronas**

Otra forma de verlo es que como se tiene que hacer una multiplicación matricial de las entradas con la primera capa de la red, si la matriz de entradas tiene un tamaño de 32x13, la matriz que representa las neuronas de la primera capa tiene que tener un tamaño de 13xM. Ya que en una multiplicación matricial, el tamaño de las matrices que se multiplican tienen que ser AxB y BxC, es decir, la dimensión de en medio de las dos matrices tiene que ser la misma

Por otro lado, el mismo batch a la salida tiene un target con este tamaño

In [16]:
target.shape

torch.Size([32, 1])

32 es el tamaño del batch size, pero solo hay 1 target, que es el precio de las casas, por lo que **a la salida queremos que haya solo 1 neurona**

In [17]:
num_inputs = parameters.shape[1]
num_outputs = target.shape[1]
model = BostonNeuralNetwork(num_inputs, num_outputs)

model

BostonNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=100, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=20, out_features=1, bias=True)
  )
)

Primero cogemos un batch del dataloader y se lo metemos a la red a ver si funciona y la hemos definido bien

In [18]:
preds = model(parameters)
preds.shape

torch.Size([32, 1])

Si se puede se manda la red a la GPU

In [19]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model.to(device)

Using cuda device


BostonNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=100, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=20, out_features=1, bias=True)
  )
)

Ahora volvemos a probar a meterle un batch

In [20]:
parameters_gpu = parameters.to(device)
preds = model(parameters_gpu)
preds.shape

torch.Size([32, 1])

## Función de pérdida y optimizador

Definimos una función de pérdida y un optimizador. Como hemos visto en el cuaderno de las funciones de pérdida, como no es un problema de clasificación, la mejor función de pérdida para este problema es el `MSE`

In [21]:
LR = 5e-4

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)


## Ciclo de entrenamiento

Entrenamos la red

In [22]:
num_prints = 4

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % int(len(dataloader)/(num_prints-1)) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val_loop(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    test_loss = 0
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

Entrenamos

In [23]:
epochs = 14
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
    val_loop(val_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 592.715576  [    0/  404]
loss: 631.924072  [  128/  404]
loss: 649.493164  [  256/  404]
loss: 459.372650  [  240/  404]
Test Error: 
 Avg loss: 537.749939 

Epoch 2
-------------------------------
loss: 373.319611  [    0/  404]
loss: 427.977203  [  128/  404]
loss: 399.665833  [  256/  404]
loss: 421.489258  [  240/  404]
Test Error: 
 Avg loss: 452.626984 

Epoch 3
-------------------------------
loss: 403.540955  [    0/  404]
loss: 298.983490  [  128/  404]
loss: 393.918213  [  256/  404]
loss: 278.953186  [  240/  404]
Test Error: 
 Avg loss: 364.763123 

Epoch 4
-------------------------------
loss: 299.536438  [    0/  404]
loss: 343.799194  [  128/  404]
loss: 441.735138  [  256/  404]
loss: 130.662567  [  240/  404]
Test Error: 
 Avg loss: 282.768463 

Epoch 5
-------------------------------
loss: 198.982742  [    0/  404]
loss: 189.268021  [  128/  404]
loss: 170.600098  [  256/  404]
loss: 262.904541  [  240/  404]
Test Error: 

Vamos a meterle un dato del dataset de validación a ver qué tal lo hace la red

In [24]:
valid_parameters, valid_target = next(iter(val_dl))

predictions = model(valid_parameters.to(device))

In [27]:
print(f"para todo el batch de validación")
for i in range(len(valid_target)):
    print(f"\t se esperaba target: {valid_target[i].item():>7f}, y se ha predicho: {predictions[i].item():>7f}")

para todo el batch de validación
	 se esperaba target: 20.000000, y se ha predicho: 21.370050
	 se esperaba target: 18.600000, y se ha predicho: 21.537910
	 se esperaba target: 22.799999, y se ha predicho: 21.562872
	 se esperaba target: 10.900000, y se ha predicho: 21.077694
	 se esperaba target: 20.400000, y se ha predicho: 21.565390
	 se esperaba target: 37.900002, y se ha predicho: 21.536257
	 se esperaba target: 21.700001, y se ha predicho: 21.558929
	 se esperaba target: 10.400000, y se ha predicho: 21.415024
	 se esperaba target: 20.299999, y se ha predicho: 21.551126
	 se esperaba target: 11.300000, y se ha predicho: 21.363111
	 se esperaba target: 23.700001, y se ha predicho: 21.565447
	 se esperaba target: 23.200001, y se ha predicho: 21.567780
	 se esperaba target: 25.000000, y se ha predicho: 21.565424
	 se esperaba target: 21.900000, y se ha predicho: 21.352701
	 se esperaba target: 7.000000, y se ha predicho: 21.343679
	 se esperaba target: 8.800000, y se ha predicho: 21.

Vemos que a la red le falta más entrenamiento porque a la salida casi siempre predice en torno al mismo valor. Pero ahora no vamos a buscar la mejor red posible, eso lo haremos al final del tema